In [1]:
# importing packages

from bokeh.io import output_notebook, output_file, curdoc, push_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, Slider, ColumnDataSource, CustomJS, LabelSet, Label
from bokeh.layouts import row, column, gridplot, widgetbox
from bokeh.models.widgets import Panel, Tabs, Button
from bokeh.models.glyphs import Text
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

import matplotlib.pyplot as mplt
import matplotlib.lines as mlines
import time

output_notebook()

import numpy as np
import pandas as pd
from itertools import permutations
from gurobipy import *

Loading BokehJS ...

In [2]:
gurobi.version()

(8, 1, 0)

In [3]:
# settings

R = 9 # number of matrices max:8
M = 8 # number of rows max:20
N = 12 # number of columns max:20
K = 10 # number of zones
step = 0.05

S_i = np.array(range(0,M+1,1))
S_j = np.array(range(0,N+1,1))

In [4]:
xls = pd.ExcelFile('data.xlsx')
df = [None]*9
df[0] = pd.read_excel(xls, 'Water Pipe (Cold Hot)', header=None)
df[1] = pd.read_excel(xls, 'High Conduit', header=None)
df[2] = pd.read_excel(xls, 'Rectangular Duct', header=None)
df[3] = pd.read_excel(xls, 'CAV-VAV', header=None)
df[4] = pd.read_excel(xls, 'M-Pipe-Wet & Connections', header=None)
df[5] = pd.read_excel(xls, 'Round Duct', header=None)
df[6] = pd.read_excel(xls, 'Cable Tray', header=None)
df[7] = pd.read_excel(xls, 'Conduit Runs', header=None)
df[8] = pd.read_excel(xls, 'Electrical CAV-VAV', header=None)

In [5]:
b = [[]] * R
for r in range(R):
    b[r] = np.array(df[r])

In [6]:
# parameters

a = [[]] * R
for r in range(R):
    a[r] = np.zeros((M, N+1))
    
for r in range(R):
    for i in range(M):
        for j in range(1, N+1):
            a[r][i][j] = a[r][i][j-1] + b[r][i][j-1]
            
Si = np.array(range(0,M,1))
Sj = np.array(range(0,N+1,1))
Sk = np.array(range(0,K+1,1))
Sr = np.array(range(0,R,1))

cost = {}
for i1 in S_i[:-1]:
    for j1 in S_j[:-1]:
        for i2 in S_i[i1+1:]:
            for j2 in S_j[j1+1:]:
                tmp = np.zeros(R)
                for r in range(R):
                    for i in range(i1,i2):
                        tmp[r] += a[r][i,j2] - a[r][i,j1]
                cost[i1,j1,i2,j2] = tmp.max()

In [7]:
lb = 0
for r in range(R):
    tmp = b[r].sum()
    if tmp > lb:
        lb = tmp
lb = lb/K

In [8]:
def draw_partitions(M, N, V):
    # drawing Matrix
    col = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',\
           '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',\
           '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

    plt = figure(plot_width=500, plot_height=round(500*M/N))

    for k in range(K):
        plt.quad(top=[V[k][0]], bottom=[V[k][2]], left=[V[k][1]], right=[V[k][3]], color=col[k%20], alpha = 0.4)

    for i in range(0,M+1):
        plt.line([0, N], [i, i], line_color='black', line_width=1, alpha = 1)
    for i in range(0,N+1):
        plt.line([i, i], [0, M], line_color='black', line_width=1, alpha = 1)
    show(plt)
    return

In [9]:
TimeTotal = time.time()
cont = True
it = 1
print('**** {} ****'.format(lb))

# ------------------------------- 1
Time1 = time.time()
# model
BCP = Model('BCP_q model')
# variables
x = BCP.addVars(S_i, S_j, S_i, S_j, lb=0.0, ub=1.0, vtype='B', name="X")
OBJ = BCP.addVar(lb=0.0, vtype='C', name="OBJECTIVE")
BCP.update()
print('1 - Time = {}'.format(round(time.time()-Time1, 1)))

# ------------------------------- 2
Time = time.time()
# constraint 1
for i1 in S_i:
    for j1 in S_j:
        for i2 in S_i[:i1+1]:
            for j2 in S_j:
                BCP.remove(x[i1,j1,i2,j2])
for i1 in S_i:
    for j1 in S_j:
        for i2 in S_i[i1+1:]:
            for j2 in S_j[:j1+1]:
                BCP.remove(x[i1,j1,i2,j2])
BCP.update()
print('2 - Time = {}'.format(round(time.time()-Time, 1)))

# ------------------------------- 4
Time = time.time()
# constraint 3
BCP.addConstr((
    quicksum(x[i1,j1,i2,j2]
             for i1 in S_i[:-1]
             for j1 in S_j[:-1]
             for i2 in S_i[i1+1:]
             for j2 in S_j[j1+1:]) == K),
    name = "# of zones")
BCP.update()
print('4 - Time = {}'.format(round(time.time()-Time, 1)))

# ------------------------------- 5
Time = time.time()
BCP.addConstrs((quicksum(x[i1,j1,i2,j2]
                        for i1 in S_i[:I+1]
                        for j1 in S_j[:J+1]
                        for i2 in S_i[I+1:]
                        for j2 in S_j[J+1:]) == 1
              for I in S_i[:-1]
              for J in S_j[:-1]),
              name = "single cell cover")
BCP.update()
print('5 - Time = {}'.format(round(time.time()-Time, 1)))

# ------------------------------- 6
Time = time.time()
# constraint Obj
BCP.addConstrs((
    OBJ >= x[i1,j1,i2,j2] * cost[i1,j1,i2,j2]
    for i1 in S_i
    for j1 in S_j
    for i2 in S_i[i1+1:]
    for j2 in S_j[j1+1:]),
    name = "OBJ on Sum of the partition")

BCP.update()
print('6 - Time = {}'.format(round(time.time()-Time, 1)))
print('\nInitializing - Time = {}\n'.format(round(time.time()-Time1, 1)))

while cont:
    # ------------------------------- 3
    Time2 = time.time()
    lb_const = BCP.addConstrs((
                x[i1,j1,i2,j2] == 0
                for i1 in S_i
                for j1 in S_j
                for i2 in S_i[i1+1:]
                for j2 in S_j[j1+1:]
                if cost[i1,j1,i2,j2] > lb),
                name = "UB")
    BCP.update()
    print('3 - Time = {}'.format(round(time.time()-Time2, 1)))

    # ------------------------------- 7
    Time = time.time()
    # objective
    BCP.setObjective(OBJ, GRB.MINIMIZE)
    # solution
    BCP.optimize()
    print('\n7 - Time = {}'.format(round(time.time()-Time, 1)))
    
    # ------------------------------- closing    
    if BCP.Status == 2:
        cont = False
        print('\nIteration {} - Time = {}\n\n\n'.format(it, round(time.time()-Time2, 1)))
    else:
        lb = lb * (1+step)
        BCP.remove(lb_const)
        print('**** {} ****'.format(lb))    
        print('\nIteration {} - Time = {}\n\n\n'.format(it, round(time.time()-Time2, 1)))
        it += 1

ttime = time.time()-TimeTotal
print('\n\n\nTotal - Time = {}     Obj = {}'.format(round(ttime, 1), round(BCP.ObjVal, 2)))

**** 1.79079 ****
Academic license - for non-commercial use only
1 - Time = 0.2
2 - Time = 0.0
4 - Time = 0.0
5 - Time = 0.1
6 - Time = 0.1

Initializing - Time = 0.4

3 - Time = 0.1
Optimize a model with 4699 rows, 2809 columns and 53893 nonzeros
Variable types: 1 continuous, 2808 integer (2808 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 3593 rows and 1794 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 40 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

7 - Time = 0.0
**** 1.8803295000000002 ****

Iteration 1 - Time = 0.1



3 - Time = 0.1
Optimize a model with 4656 rows, 2809 columns and 53850 nonzeros
Variable types: 1 continuous, 2808 integer (2808 binary)
Coefficient statistics:
  Matrix range     [2e-02, 2e+01]
  Objective range  

In [10]:
gurobi.version()

(8, 1, 0)

In [11]:
# 1%     Obj = 54383.0      Time = 89.3       iterations = 11
# 3%     Obj = 54383.0      Time = 55.4       iterations = 5
# 5%     Obj = 54383.0      Time = 35.0       iterations = 3

In [12]:
V = []
for i1 in S_i[:-1]:
    for j1 in S_j[:-1]:
        for i2 in S_i[i1+1:]:
            for j2 in S_j[j1+1:]:
                if x[i1,j1,i2,j2].X > 0.001:
                    print(i1,j1,i2,j2)
                    V.append((i1,j1,i2,j2))
                    
print('\nObj = {}      Time = {}       iterations = {}'.format(round(BCP.ObjVal, 2), round(ttime, 1), it))
draw_partitions(M, N, V)

0 0 8 2
0 2 2 7
0 7 2 11
0 11 8 12
2 2 3 7
2 7 4 11
3 2 4 7
4 2 5 11
5 2 8 7
5 7 8 11

Obj = 2.1      Time = 1.0       iterations = 5


In [13]:
R_name = ['Water Pipe (Cold Hot)', 'High Conduit', 'Rectangular Duct', 'CAV-VAV', 'M-Pipe-Wet & Connections',\
          'RoundDuct', 'CableTray', 'Conduit Runs', 'Electrical CAV-VAV']

col_rct = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0',\
       '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8',\
       '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

col = ['#000075','#42d4f4','#f58231','#8A2BE2','#A52A2A','#f032e6','#4363d8','#808000', 'green']

takt = np.zeros(R*K).reshape(K,R)
for i, v in enumerate(V):
    for t in range(R):
        tmp = 0
        for row in range(v[0],v[2]):
            tmp += a[t][row,v[3]] - a[t][row,v[1]]
            takt[i][t] = tmp

partitions = []
for i in range(1,K+1):
    partitions.append("Partition_" + str(i))

data = {}
data['prtitions'] = partitions
for r in range(R):
    data[R_name[r]] = []
    for k in range(K):
        data[R_name[r]].append(takt[k,r])

w = 2.8/(5*R-1)
source = ColumnDataSource(data=data)

p = figure(x_range=partitions, y_range=(0, BCP.ObjVal*1.5), plot_height=350, plot_width=1250,\
           title="TAKT planning result", tools="save")

for k in range(K):
    p.quad(top=[BCP.ObjVal*1.3], bottom=[0], left=[k], right=[k+1], color=col_rct[k], alpha = 0.2)    

for r in range(R):
    p.vbar(x=dodge('prtitions', w*5/4*r-0.35+w/2, range=p.x_range), top=R_name[r], width=w, source=source,
       color=col[r], legend=value(R_name[r]))

p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)